In [18]:
import os

import numpy as np
import pandas as pd
import torch

# Feature Concatenate

In [19]:
valence_lstm_features = np.load('valence_lstm_features.npy', allow_pickle=True).tolist()
arousal_lstm_features = np.load('arousal_lstm_features.npy', allow_pickle=True).tolist()
emotion_lstm_features = np.load('emotion_lstm_features.npy', allow_pickle=True).tolist()
labels = np.load('labels.npy', allow_pickle=True)

In [20]:
l = labels.tolist()
l = [i.tolist() for i in labels]
l_list = sum(l,[])
print(l_list.count(1))
print(l_list.count(0))

41591
78639


In [25]:
# convert each element of the list to a tensor
valence_lstm_tensors = [torch.tensor(arr) for arr in valence_lstm_features]
arousal_lstm_tensors = [torch.tensor(arr) for arr in arousal_lstm_features]
emotion_lstm_tensors = [torch.tensor(arr) for arr in emotion_lstm_features]

In [26]:
features_concatenate_list = []
for valence_feature, arousal_feature, emotion_feature in zip(valence_lstm_tensors, arousal_lstm_tensors, emotion_lstm_tensors):
    concated_feature = torch.cat((valence_feature, arousal_feature, emotion_feature), dim=1)
    features_concatenate_list.append(concated_feature)

# Dataset, DataLoader

In [27]:
from torch.utils.data import DataLoader, Dataset

In [28]:
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [29]:
# Create an instance of dataset
md = MyDataset(features_concatenate_list,labels)

In [30]:
dataloader = DataLoader(md)

# Model

In [31]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        
        return out

In [32]:
import torch
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode the hidden state of the last time step
        output = self.fc(out[:, -1, :])
        
        # Apply sigmoid activation function to output
        output = self.sigmoid(output)
        
        return output, out[:, -1, :]


In [149]:
import torch
import torch.nn as nn

class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # Set initial hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate GRU
        out, _ = self.gru(x, h0)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        
        return out


# Training

In [33]:
num_epochs = 100
learning_rate = 0.001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size = 224
hidden_size = 128
num_layers = 3
output_size = 1

model = LSTMClassifier(input_size, hidden_size, num_layers, output_size).to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [34]:
from tqdm.notebook import tqdm_notebook
# LSTM
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in tqdm_notebook(dataloader, total=len(dataloader), desc='Training'):
        if inputs.shape[1] != labels.shape[1] : continue
        
        inputs = inputs.transpose(0,1).to(device)
        labels = labels.squeeze(0).to(device)
        
        optimizer.zero_grad()

        # Forward pass
        outputs,out = model(inputs)
        outputs = outputs.squeeze(1)
        loss = criterion(outputs, labels.float())

        # Backward and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, running_loss/len(dataloader)))


Training:   0%|          | 0/556 [00:00<?, ?it/s]

Epoch [1/100], Loss: 0.6268


Training:   0%|          | 0/556 [00:00<?, ?it/s]

Epoch [2/100], Loss: 0.6223


Training:   0%|          | 0/556 [00:00<?, ?it/s]

...

Epoch [98/100], Loss: 0.6195


Training:   0%|          | 0/556 [00:00<?, ?it/s]

Epoch [99/100], Loss: 0.6195


Training:   0%|          | 0/556 [00:00<?, ?it/s]

Epoch [100/100], Loss: 0.6195


In [35]:
outputs

tensor([0.3310, 0.3337, 0.3208, 0.3295, 0.3299, 0.3313, 0.3295, 0.3298, 0.3302,
        0.3334, 0.3298, 0.3313, 0.3307, 0.3294, 0.3320, 0.3301, 0.3331, 0.3294,
        0.3290, 0.3307, 0.3298, 0.3326, 0.3304, 0.3308, 0.3298, 0.3300, 0.3301,
        0.3307, 0.3311, 0.3298, 0.3296, 0.3315, 0.3287, 0.3318, 0.3302, 0.3297,
        0.3198, 0.3296, 0.3263, 0.3224], device='cuda:0',
       grad_fn=<SqueezeBackward1>)

In [36]:
PATH = 'concatenate_lstm_model.pt'

# 모델 저장
torch.save(model.state_dict(), PATH)

In [37]:
%pwd

'/workspace/EmotionShortForm/lstm'

In [151]:
# GRU
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in tqdm_notebook(dataloader, total=len(dataloader), desc='Training'):
        if inputs.shape[1] != labels.shape[1] : continue
        
        inputs = inputs.transpose(0,1).to(device)
        labels = labels.squeeze(0).to(device)
        
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs).squeeze(1)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, running_loss/len(dataloader)))


Training:   0%|          | 0/556 [00:00<?, ?it/s]

Epoch [1/10], Loss: 0.6252


Training:   0%|          | 0/556 [00:00<?, ?it/s]

Epoch [2/10], Loss: 0.6223


Training:   0%|          | 0/556 [00:00<?, ?it/s]

...

Epoch [8/10], Loss: 0.6216


Training:   0%|          | 0/556 [00:00<?, ?it/s]

Epoch [9/10], Loss: 0.6216


Training:   0%|          | 0/556 [00:00<?, ?it/s]

Epoch [10/10], Loss: 0.6216


In [152]:
outputs

tensor([-0.7756, -0.7756, -0.7756, -0.7756, -0.7756, -0.7756, -0.7756, -0.7756,
        -0.7756, -0.7756, -0.7756, -0.7756, -0.7756, -0.7756, -0.7756, -0.7756,
        -0.7756, -0.7756, -0.7756, -0.7756, -0.7756, -0.7756, -0.7756, -0.7756,
        -0.7756, -0.7756, -0.7756, -0.7756, -0.7756, -0.7756, -0.7756, -0.7756,
        -0.7756, -0.7756, -0.7756, -0.7756, -0.7756, -0.7756, -0.7756, -0.7756],
       device='cuda:0', grad_fn=<SqueezeBackward1>)

# Feature Extract

In [38]:
model = LSTMClassifier(input_size, hidden_size, num_layers, output_size).to(device)

# 모델 불러오기
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [40]:
from tqdm.notebook import tqdm_notebook

concatenate_features = []

count = 1
for inputs, labels in tqdm_notebook(dataloader,total=len(dataloader)):
    with torch.no_grad():
        model.eval()
#         if inputs.shape[1] != labels.shape[1] : continue
        
        inputs = inputs.transpose(0,1).to(device)
        labels = labels.squeeze(0).to(device)

        # Forward pass
        outputs,out = model(inputs)
        count += 1
        concatenate_features.append(out)

  0%|          | 0/556 [00:00<?, ?it/s]

In [43]:
len(concatenate_features), concatenate_features[0].shape

(556, torch.Size([250, 128]))

In [44]:
data_array = np.array(concatenate_features)

np.save('concatenate_features_array.npy',data_array)

In [45]:
x = np.load('concatenate_features_array.npy',allow_pickle=True)
x.shape

(556,)

In [46]:
x_list = x.tolist()
x_list[0].shape

torch.Size([250, 128])